# Airlift planning

You are in charge of determining which subset of the following requirements should be shipped on the next C-17 to another base:

| Requirement      | Capability Value | Weight (tons) | Volume ($\text{m}^3$) |
|:-----------------|:-----------------|:--------------|:-------------|
| Large munitions  | 50               | 43            | 250          |
| Small munitions  | 30               | 17            | 130          |
| Food             | 80               | 26            | 370          |
| Medical supplies | 40               | 4             | 180          |
| Repair parts     | 70               | 35            | 400          |

The C-17 has a weight capacity of 80 tons, and a volume capacity of 700 $\text{m}^3$. The goal is to maximize the total capability value of the requirements shipped.


* Formulate this problem as a dynamic program by giving its shortest/longest path representation.


* Solve this dynamic program. 
    - What is the maximum possible total capability value?
    - Which requirements should be put on the plane to achieve this maximum total capability value?

![Longest path representation of DP](img/airlift_dp.png)

In [1]:
# Write your code here
# Import networkx and bellmanford
import networkx as nx
import bellmanford as bf

# Create variables for weight and volume capacity
MAX_WEIGHT = 80
MAX_VOLUME = 700 

# Create lists for capability values, weights, volumes
value = [50, 30, 80, 40, 70]
weight = [43, 17, 26, 4, 35]
volume = [250, 130, 370, 180, 400]

# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes (t, n1, n2)
#   t = stage, consider requirement t
#   n1 = remaining weight
#   n2 = remaining volume
# 
# t = 0: large munitions
# t = 1: small munitions
# t = 2: food
# t = 3: medical supplies
# t = 4: repair parts
# t = 5: end of decision process
for t in range(0, 6):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            G.add_node((t, n1, n2))

# Add end node
G.add_node("end")

# Add edges corresponding to decisions
# Remember we're maximizing, so the lengths should be negative of the values
for t in range(0, 5):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            # Take requirement t, check if it fits first
            if n1 - weight[t] >= 0:
                if n2 - volume[t] >= 0:
                    G.add_edge((t, n1, n2), (t + 1, n1 - weight[t], n2 - volume[t]), 
                               length=-value[t])

            # Don't take requirement t
            G.add_edge((t, n1, n2), (t + 1, n1, n2), length=0)

# Add edges from last stage to end node
for n1 in range(0, MAX_WEIGHT + 1):
    for n2 in range(0, MAX_VOLUME + 1):
        G.add_edge((5, n1, n2), "end", length = 0)

# Solve the DP/shortest path problem using the Bellman-Ford algorithm
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, MAX_WEIGHT, MAX_VOLUME), 
                                                target="end", weight="length")

print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -150
Shortest path: [(0, 80, 700), (1, 80, 700), (2, 63, 570), (3, 37, 200), (4, 33, 20), (5, 33, 20), 'end']


* The maximum possible total capability value is 150.

* To achieve this maximum total capability value, we should put the small munitions, food, and medical supplies on the C-17.